<p style = "font-size : 50px; color : 'blue' ; font-family : 'Arial'; text-align : center; background-color : #bedcfa; border-radius: 5px 5px;"><strong>Retrasos de vuelos </strong></p>

## Parte 1 - Entender el problema
El principal objetivo de negocio que se quiere abordar es predecir el retraso de vuelos comerciales, usando datos históricos de vuelos y de climatología. 

Para esto tenemos 2 conjuntos de datos: por un lado tenemos la información de vuelos y por el otro la información climatológica. 

El primer conjunto de datos contiene datos de retrasos de vuelos para el período de abril a octubre de 2013 especificando información sobre origen, destino, fecha, hora.

Los datos meteorológicos representan observaciones de las estaciones meteorológicas del aeropuerto, que cubren el mismo período de tiempo de abril a octubre de 2013.

<center><img src="https://www.webintravel.com/wp-content/uploads/2020/05/den-belitsky-GettyImages-854673918-scaled.jpg" width="700"></center>

#### Importamos las librerias necesarias para el ejercicio

In [40]:
!pip freeze

absl-py==0.15.0
adjustText==0.8
aenum==2.2.6
aiohttp==3.8.6
aiohttp-retry==2.8.3
aiosignal==1.3.1
alembic==1.12.1
amqp==5.1.1
aniso8601==9.0.1
annotated-types==0.5.0
ansi2html==1.8.0
antlr4-python3-runtime==4.9.3
anyio==3.6.2
apache-libcloud==3.3.1
appdirs==1.4.4
apyori==1.1.2
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
asgiref==3.3.1
asttokens==2.0.8
astunparse==1.6.3
async-timeout==4.0.3
asyncssh==2.14.0
atomicwrites==1.4.0
atpublic==4.0
attrs==23.1.0
auto-labeling-pipeline==0.1.15
aws-cdk.assets==1.101.0
aws-cdk.aws-apigateway==1.101.0
aws-cdk.aws-applicationautoscaling==1.101.0
aws-cdk.aws-autoscaling==1.101.0
aws-cdk.aws-autoscaling-common==1.101.0
aws-cdk.aws-autoscaling-hooktargets==1.101.0
aws-cdk.aws-batch==1.101.0
aws-cdk.aws-certificatemanager==1.101.0
aws-cdk.aws-cloudformation==1.101.0
aws-cdk.aws-cloudfront==1.101.0
aws-cdk.aws-cloudwatch==1.101.0
aws-cdk.aws-codebuild==1.101.0
aws-cdk.aws-codecommit==1.101.0
aws-cdk.aws-codeguruprofiler==1.101.0
aws-cdk.aws-codepipe

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay
#from ydata_profiling import ProfileReport
import joblib
plt.style.use("fivethirtyeight")
%matplotlib inline

## Parte 2 - Comprensión de los datos

In [2]:
# Cargar los datos del clima 
df_clima=pd.read_csv('Weather Dataset.csv') 
#Muestra las 10 primeras filas del dataset
df_clima.head(10) 

C:\Users\dhoyoso\AppData\Local\Temp\ipykernel_9896\2199647736.py:2: DtypeWarning: Columns (7,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_clima=pd.read_csv('Weather Dataset.csv')


,AirportID,Year,Month,Day,Time,TimeZone,SkyCondition,Visibility,WeatherType,DryBulbFarenheit,...,WindSpeed,WindDirection,ValueForWindCharacter,StationPressure,PressureTendency,PressureChange,SeaLevelPressure,RecordType,HourlyPrecip,Altimeter
0,14843,2013,4,1,56,-4,FEW018 SCT044 BKN070,10.00,-RA,76,...,13,080,,30.06,,,30.06,AA,T,30.07
1,14843,2013,4,1,156,-4,FEW037 SCT070,10.00,,76,...,10,090,,30.05,6,017,30.05,AA,,30.06
2,14843,2013,4,1,256,-4,FEW037 SCT070,10.00,,76,...,9,100,,30.03,,,30.03,AA,,30.04
3,14843,2013,4,1,356,-4,FEW025 SCT070,10.00,,76,...,9,100,,30.02,,,30.03,AA,,30.03
4,14843,2013,4,1,456,-4,FEW025,10.00,,76,...,7,110,,30.03,5,004,30.04,AA,,30.04
5,14843,2013,4,1,556,-4,FEW025 SCT080,10.00,,76,...,7,100,,30.04,,,30.05,AA,,30.05
6,14843,2013,4,1,656,-4,FEW028 BKN080,10.00,,77,...,9,110,,30.07,,,30.07,AA,,30.08
7,14843,2013,4,1,756,-4,FEW028 BKN080,10.00,,79,...,13,100,,30.09,3,020,30.10,AA,,30.10
8,14843,2013,4,1,856,-4,FEW030 BKN080,10.00,,82,...,14,100,21,30.11,,,30.11,AA,,30.12
9,14843,2013,4,1,956,-4,SCT035 BKN090,10.00,,83,...,16,090,23,30.11,,,30.12,AA,,30.12


##### Diccionario de datos
- Year, Month, Day, Time, TimeZone – Fecha y hora del vuelo
- AirportID – Número de Identificación para un único aeropuerto
- Un set de variables metereológicas específicas: SkyCondition, Visibility, WeatherType, DryBulbFarenheit, DryBulbCelsius, StationPressure, 
PressureTendency, PressureChange, SeaLevelPressure, RecordType, HourlyPrecip, Altimeter,….

In [3]:
#Cargar los datos de los vuelos 
df_vuelos = pd.read_csv('Flight Delays Data.csv')
df_vuelos.head(10)

,Year,Month,DayofMonth,DayOfWeek,Carrier,OriginAirportID,DestAirportID,CRSDepTime,DepDelay,DepDel15,CRSArrTime,ArrDelay,ArrDel15,Cancelled
0,2013,4,19,5,DL,11433,13303,837,-3.0,0.0,1138,1.0,0.0,0.0
1,2013,4,19,5,DL,14869,12478,1705,0.0,0.0,2336,-8.0,0.0,0.0
2,2013,4,19,5,DL,14057,14869,600,-4.0,0.0,851,-15.0,0.0,0.0
3,2013,4,19,5,DL,15016,11433,1630,28.0,1.0,1903,24.0,1.0,0.0
4,2013,4,19,5,DL,11193,12892,1615,-6.0,0.0,1805,-11.0,0.0,0.0
5,2013,4,19,5,DL,10397,15016,1726,-1.0,0.0,1818,-19.0,0.0,0.0
6,2013,4,19,5,DL,15016,10397,1900,0.0,0.0,2133,-1.0,0.0,0.0
7,2013,4,19,5,DL,10397,14869,2145,15.0,1.0,2356,24.0,1.0,0.0
8,2013,4,19,5,DL,10397,10423,2157,33.0,1.0,2333,34.0,1.0,0.0
9,2013,4,19,5,DL,11278,10397,1900,323.0,1.0,2055,322.0,1.0,0.0


##### Diccionario de datos
- Información del vuelo (Calendario): Año – Mes – Día
- Carrier- Código usado comunmente para identificar una aerolínea
- Aeropuerto Origen – Destino – Codigo de identificación del aeropuerto
- CRS DepTme - CRSArr Time Los tiempos CRS de salida y llegada, usando hora local (hhmm)
- Dep Delay- Arr delay Diferencia en minutos entre el tiempos de salida y llegada programados y los actuales. 
- DepDel15 - ArrDel15 Un valor boleano que indica si los tiempos de salida o llegada tienen un retraso de más de 15 minutos.
- Cancelled - Un valor booleano que indica si el vuelo fue cancelado (1=Vuelo cancelado)

In [4]:
#Información sobre el set de datos
df_clima.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406516 entries, 0 to 406515
Data columns (total 26 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   AirportID              406516 non-null  int64 
 1   Year                   406516 non-null  int64 
 2   Month                  406516 non-null  int64 
 3   Day                    406516 non-null  int64 
 4   Time                   406516 non-null  int64 
 5   TimeZone               406516 non-null  int64 
 6   SkyCondition           406516 non-null  object
 7   Visibility             406516 non-null  object
 8   WeatherType            406516 non-null  object
 9   DryBulbFarenheit       406516 non-null  object
 10  DryBulbCelsius         406516 non-null  object
 11  WetBulbFarenheit       406516 non-null  object
 12  WetBulbCelsius         406516 non-null  object
 13  DewPointFarenheit      406516 non-null  object
 14  DewPointCelsius        406516 non-null  object
 15  

In [5]:
#Información sobre el set de datos
df_vuelos.info()   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1006472 entries, 0 to 1006471
Data columns (total 14 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   Year             1006472 non-null  int64  
 1   Month            1006472 non-null  int64  
 2   DayofMonth       1006472 non-null  int64  
 3   DayOfWeek        1006472 non-null  int64  
 4   Carrier          1006472 non-null  object 
 5   OriginAirportID  1006472 non-null  int64  
 6   DestAirportID    1006472 non-null  int64  
 7   CRSDepTime       1006472 non-null  int64  
 8   DepDelay         994663 non-null   float64
 9   DepDel15         994663 non-null   float64
 10  CRSArrTime       1006472 non-null  int64  
 11  ArrDelay         993999 non-null   float64
 12  ArrDel15         1006472 non-null  float64
 13  Cancelled        1006472 non-null  float64
dtypes: float64(5), int64(8), object(1)
memory usage: 107.5+ MB


## Parte 3 Preparación de datos

### 3.1 Transformación de variables

¿ Qué variables de los datasets no están en el tipo correcto? ¿Qué podemos hacer con los ids?

In [6]:
# Vamos primero a transformar los tipos de datos de las variables del dataset de clima
# Transforma una variable numérica a una categórica (Tipo String)
df_clima['AirportID'] = df_clima['AirportID'].astype(str)

#Transforma el tipo de datos a variables numéricas (enteros y decimales)
df_clima['Visibility'] = pd.to_numeric(df_clima['Visibility'], errors='coerce')
df_clima['DryBulbFarenheit'] = pd.to_numeric(df_clima['DryBulbFarenheit'], errors='coerce')
df_clima['DryBulbCelsius'] = pd.to_numeric(df_clima['DryBulbCelsius'], errors='coerce')
df_clima['WetBulbFarenheit'] = pd.to_numeric(df_clima['WetBulbFarenheit'], errors='coerce')
df_clima['WetBulbCelsius'] = pd.to_numeric(df_clima['WetBulbCelsius'], errors='coerce')
df_clima['DewPointFarenheit'] = pd.to_numeric(df_clima['DewPointFarenheit'], errors='coerce')
df_clima['DewPointCelsius'] = pd.to_numeric(df_clima['DewPointCelsius'], errors='coerce')
df_clima['RelativeHumidity'] = pd.to_numeric(df_clima['RelativeHumidity'], errors='coerce')
df_clima['WindSpeed'] = pd.to_numeric(df_clima['WindSpeed'], errors='coerce')
df_clima['WindDirection'] = pd.to_numeric(df_clima['WindDirection'], errors='coerce')
df_clima['ValueForWindCharacter'] = pd.to_numeric(df_clima['ValueForWindCharacter'], errors='coerce')
df_clima['StationPressure'] = pd.to_numeric(df_clima['StationPressure'], errors='coerce')
df_clima['PressureTendency'] = pd.to_numeric(df_clima['PressureTendency'], errors='coerce')
df_clima['PressureChange'] = pd.to_numeric(df_clima['PressureChange'], errors='coerce')
df_clima['SeaLevelPressure'] = pd.to_numeric(df_clima['SeaLevelPressure'], errors='coerce')
df_clima['Altimeter'] = pd.to_numeric(df_clima['Altimeter'], errors='coerce')

In [7]:
df_clima.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406516 entries, 0 to 406515
Data columns (total 26 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   AirportID              406516 non-null  object 
 1   Year                   406516 non-null  int64  
 2   Month                  406516 non-null  int64  
 3   Day                    406516 non-null  int64  
 4   Time                   406516 non-null  int64  
 5   TimeZone               406516 non-null  int64  
 6   SkyCondition           406516 non-null  object 
 7   Visibility             406408 non-null  float64
 8   WeatherType            406516 non-null  object 
 9   DryBulbFarenheit       406227 non-null  float64
 10  DryBulbCelsius         406227 non-null  float64
 11  WetBulbFarenheit       405019 non-null  float64
 12  WetBulbCelsius         405019 non-null  float64
 13  DewPointFarenheit      406135 non-null  float64
 14  DewPointCelsius        406135 non-nu

In [8]:
# Ahora vamos a transformar los tipos de datos de las variables del dataset de vuelos

#Transformamos los números de identificación a tipo string
df_vuelos['OriginAirportID'] = df_vuelos['OriginAirportID'].astype(str)
df_vuelos['DestAirportID'] = df_vuelos['DestAirportID'].astype(str)

# Transformamos las variables a tipo booleano cuando el 1 o 0 indican verdadero o falso
df_vuelos['DepDel15'] = df_vuelos['DepDel15'].astype(bool)
df_vuelos['ArrDel15'] = df_vuelos['ArrDel15'].astype(bool)
df_vuelos['Cancelled'] = df_vuelos['Cancelled'].astype(bool)

In [9]:
df_vuelos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1006472 entries, 0 to 1006471
Data columns (total 14 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   Year             1006472 non-null  int64  
 1   Month            1006472 non-null  int64  
 2   DayofMonth       1006472 non-null  int64  
 3   DayOfWeek        1006472 non-null  int64  
 4   Carrier          1006472 non-null  object 
 5   OriginAirportID  1006472 non-null  object 
 6   DestAirportID    1006472 non-null  object 
 7   CRSDepTime       1006472 non-null  int64  
 8   DepDelay         994663 non-null   float64
 9   DepDel15         1006472 non-null  bool   
 10  CRSArrTime       1006472 non-null  int64  
 11  ArrDelay         993999 non-null   float64
 12  ArrDel15         1006472 non-null  bool   
 13  Cancelled        1006472 non-null  bool   
dtypes: bool(3), float64(2), int64(6), object(3)
memory usage: 87.3+ MB


¿ Ahora qué hacemos con las variables que nos inidican fechas u horas? Es importante que estas variables sean consistentes entre los 2 datasets para garantizar que se puede integrar la información de manera consistente

In [10]:
# Como las horas de las mediciones metereológicas no son exactamente las mismas 
#a las horas de despegue de los vuelos haremos una aproximación a la hora más cercana

# Tranformamos la variable del data frame de vuelos
df_vuelos['CRSDepTime'] = df_vuelos['CRSDepTime'].apply(lambda x: x/100)
df_vuelos['CRSDepTime'] = df_vuelos['CRSDepTime'].round(0)
df_vuelos['CRSDepTime'] 

0           8.0
1          17.0
2           6.0
3          16.0
4          16.0
           ... 
1006467     8.0
1006468    18.0
1006469    22.0
1006470    12.0
1006471    10.0
Name: CRSDepTime, Length: 1006472, dtype: float64

In [11]:
# Realice la misma operación para la variable del data frame de clima
df_clima['Time'] = df_clima['Time'].apply(lambda x: x/100)
df_clima['Time'] = df_clima['Time'].round(0)
df_clima['Time']

0          1.0
1          2.0
2          3.0
3          4.0
4          5.0
          ... 
406511    22.0
406512    22.0
406513    23.0
406514    23.0
406515    24.0
Name: Time, Length: 406516, dtype: float64

### 3.2 Tratamiento de valores faltanes

In [12]:
# Identificamos la cantidad de valores nulos en cada uno de los datasets
df_vuelos.isnull().sum().divide(len(df_vuelos)).multiply(100)

Year               0.000000
Month              0.000000
DayofMonth         0.000000
DayOfWeek          0.000000
Carrier            0.000000
OriginAirportID    0.000000
DestAirportID      0.000000
CRSDepTime         0.000000
DepDelay           1.173306
DepDel15           0.000000
CRSArrTime         0.000000
ArrDelay           1.239279
ArrDel15           0.000000
Cancelled          0.000000
dtype: float64

¿Qué estrategia para el tratamiento de valores faltantes escaogería para cada una de las variables?

In [13]:
# Elimanos las observaciones que tienen nulos en la variables DepDelay y ArrDelay 
# dado que esta es la variable y el % de faltantes es bajo

df_vuelos.dropna(subset=['DepDelay'], inplace=True)
df_vuelos.dropna(subset=['ArrDelay'], inplace=True)
df_vuelos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 993999 entries, 0 to 1006471
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Year             993999 non-null  int64  
 1   Month            993999 non-null  int64  
 2   DayofMonth       993999 non-null  int64  
 3   DayOfWeek        993999 non-null  int64  
 4   Carrier          993999 non-null  object 
 5   OriginAirportID  993999 non-null  object 
 6   DestAirportID    993999 non-null  object 
 7   CRSDepTime       993999 non-null  float64
 8   DepDelay         993999 non-null  float64
 9   DepDel15         993999 non-null  bool   
 10  CRSArrTime       993999 non-null  int64  
 11  ArrDelay         993999 non-null  float64
 12  ArrDel15         993999 non-null  bool   
 13  Cancelled        993999 non-null  bool   
dtypes: bool(3), float64(3), int64(5), object(3)
memory usage: 93.8+ MB


In [14]:
# Identificamos la cantidad de valores nulos en cada uno de los datasets
# Calculamos el % de faltantes en cada una de las variables
df_clima.isnull().sum().divide(len(df_clima)).multiply(100)

AirportID                 0.000000
Year                      0.000000
Month                     0.000000
Day                       0.000000
Time                      0.000000
TimeZone                  0.000000
SkyCondition              0.000000
Visibility                0.026567
WeatherType               0.000000
DryBulbFarenheit          0.071092
DryBulbCelsius            0.071092
WetBulbFarenheit          0.368251
WetBulbCelsius            0.368251
DewPointFarenheit         0.093723
DewPointCelsius           0.093723
RelativeHumidity          0.368251
WindSpeed                 0.036653
WindDirection             2.817355
ValueForWindCharacter    87.818684
StationPressure           0.300603
PressureTendency         96.066083
PressureChange           96.066083
SeaLevelPressure         17.932873
RecordType                0.000000
HourlyPrecip              0.000000
Altimeter                 0.027305
dtype: float64

¿Qué estrategia para el tratamiento de valores faltantes escaogería para cada una de las variables?

In [15]:
# Eliminamos las variables que tienen un alto % de faltantes
df_clima =df_clima.drop(['ValueForWindCharacter', 'PressureTendency','PressureChange','SeaLevelPressure'], axis=1)

#Eliminamos las obersvaciones que tienen alguna otro valor faltante dado que es un pequeño %
df_clima = df_clima.dropna(how='any',axis=0) 

df_clima.info()

<class 'pandas.core.frame.DataFrame'>
Index: 393636 entries, 0 to 406515
Data columns (total 22 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   AirportID          393636 non-null  object 
 1   Year               393636 non-null  int64  
 2   Month              393636 non-null  int64  
 3   Day                393636 non-null  int64  
 4   Time               393636 non-null  float64
 5   TimeZone           393636 non-null  int64  
 6   SkyCondition       393636 non-null  object 
 7   Visibility         393636 non-null  float64
 8   WeatherType        393636 non-null  object 
 9   DryBulbFarenheit   393636 non-null  float64
 10  DryBulbCelsius     393636 non-null  float64
 11  WetBulbFarenheit   393636 non-null  float64
 12  WetBulbCelsius     393636 non-null  float64
 13  DewPointFarenheit  393636 non-null  float64
 14  DewPointCelsius    393636 non-null  float64
 15  RelativeHumidity   393636 non-null  float64
 16  WindSpe

### Parte 3.3 - Integración de los datos

¿Cuáles son variables presentes en ambos conjunstos de datos?¿ De estas, qué variables debemos coincidir entre el conjunto de datos de vuelos y el de clima para garantizar una integración de datos consistente? 

In [16]:
# Integramos las 2 bases de datos en un único conjunto de datos
merged_data = df_vuelos.merge(df_clima, left_on=['Month','DayofMonth','OriginAirportID','CRSDepTime'], 
                                right_on=['Month','Day','AirportID','Time'], how='inner')
merged_data.head()

,Year_x,Month,DayofMonth,DayOfWeek,Carrier,OriginAirportID,DestAirportID,CRSDepTime,DepDelay,DepDel15,...,WetBulbCelsius,DewPointFarenheit,DewPointCelsius,RelativeHumidity,WindSpeed,WindDirection,StationPressure,RecordType,HourlyPrecip,Altimeter
0,2013,4,19,5,DL,11433,13303,8.0,-3.0,False,...,6.9,44.0,6.7,96.0,16.0,220.0,28.75,AA,T,29.45
1,2013,4,19,5,DL,11433,13303,8.0,-3.0,False,...,6.6,43.0,6.0,93.0,21.0,220.0,28.77,SP,,29.47
2,2013,4,19,5,DL,11433,11298,8.0,22.0,True,...,6.9,44.0,6.7,96.0,16.0,220.0,28.75,AA,T,29.45
3,2013,4,19,5,DL,11433,11298,8.0,22.0,True,...,6.6,43.0,6.0,93.0,21.0,220.0,28.77,SP,,29.47
4,2013,4,19,5,DL,11433,11292,8.0,-5.0,False,...,6.9,44.0,6.7,96.0,16.0,220.0,28.75,AA,T,29.45


In [17]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1029807 entries, 0 to 1029806
Data columns (total 35 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Year_x             1029807 non-null  int64  
 1   Month              1029807 non-null  int64  
 2   DayofMonth         1029807 non-null  int64  
 3   DayOfWeek          1029807 non-null  int64  
 4   Carrier            1029807 non-null  object 
 5   OriginAirportID    1029807 non-null  object 
 6   DestAirportID      1029807 non-null  object 
 7   CRSDepTime         1029807 non-null  float64
 8   DepDelay           1029807 non-null  float64
 9   DepDel15           1029807 non-null  bool   
 10  CRSArrTime         1029807 non-null  int64  
 11  ArrDelay           1029807 non-null  float64
 12  ArrDel15           1029807 non-null  bool   
 13  Cancelled          1029807 non-null  bool   
 14  AirportID          1029807 non-null  object 
 15  Year_y             1029807 non-n

### 3.4 Selección de variables

¿Qué variables quedaron duplicadas al hacer la integración de las bases de datos?
¿Qué variables están proporcionando la misma información?


In [18]:
#Primero eliminamos todas las variables duplicadas
merged_data =merged_data.drop(['Year_x', 'DayofMonth','OriginAirportID','Time'], axis=1)

# Ahora eliminamos variables que nos están proporcionando la misma información (e.g., Farenheit y Celsius)
merged_data =merged_data.drop(['DryBulbFarenheit', 'WetBulbFarenheit','DewPointFarenheit',
                               'DepDelay','ArrDelay'], axis=1)

Ahora pensemos cuáles de las variables que tenemos en nuestro dataset no nos proporcionan información relevante para hacer la predicción o son una fuga de información (esto quiere decir que son variables que no tendremos registradas al momento de hacer la predicción para nuevos datos)

In [19]:
#Eliminamos las variables que no proprocionan información útil
merged_data =merged_data.drop(['Year_y','TimeZone'], axis=1) #'DestAirportID','AirportID'

# Ahora eliminamos variables que son fuga de información
merged_data =merged_data.drop(['ArrDel15','Cancelled','CRSArrTime','CRSDepTime'], axis=1)

#Eliminamos variables que tienen el mismo valorr en todo el conjunto
merged_data =merged_data.drop(['Carrier','SkyCondition','WeatherType','HourlyPrecip'], axis=1)
merged_data.head()

,Month,DayOfWeek,DestAirportID,DepDel15,AirportID,Day,Visibility,DryBulbCelsius,WetBulbCelsius,DewPointCelsius,RelativeHumidity,WindSpeed,WindDirection,StationPressure,RecordType,Altimeter
0,4,5,13303,False,11433,19,10.0,7.2,6.9,6.7,96.0,16.0,220.0,28.75,AA,29.45
1,4,5,13303,False,11433,19,10.0,7.0,6.6,6.0,93.0,21.0,220.0,28.77,SP,29.47
2,4,5,11298,True,11433,19,10.0,7.2,6.9,6.7,96.0,16.0,220.0,28.75,AA,29.45
3,4,5,11298,True,11433,19,10.0,7.0,6.6,6.0,93.0,21.0,220.0,28.77,SP,29.47
4,4,5,11292,False,11433,19,10.0,7.2,6.9,6.7,96.0,16.0,220.0,28.75,AA,29.45


In [20]:
Record = pd.get_dummies(merged_data['RecordType'])
merged_data = pd.concat([merged_data, Record], axis = 1)
merged_data.head()

,Month,DayOfWeek,DestAirportID,DepDel15,AirportID,Day,Visibility,DryBulbCelsius,WetBulbCelsius,DewPointCelsius,RelativeHumidity,WindSpeed,WindDirection,StationPressure,RecordType,Altimeter,AA,SP,SY-MT
0,4,5,13303,False,11433,19,10.0,7.2,6.9,6.7,96.0,16.0,220.0,28.75,AA,29.45,True,False,False
1,4,5,13303,False,11433,19,10.0,7.0,6.6,6.0,93.0,21.0,220.0,28.77,SP,29.47,False,True,False
2,4,5,11298,True,11433,19,10.0,7.2,6.9,6.7,96.0,16.0,220.0,28.75,AA,29.45,True,False,False
3,4,5,11298,True,11433,19,10.0,7.0,6.6,6.0,93.0,21.0,220.0,28.77,SP,29.47,False,True,False
4,4,5,11292,False,11433,19,10.0,7.2,6.9,6.7,96.0,16.0,220.0,28.75,AA,29.45,True,False,False


In [21]:
merged_data.drop(['RecordType'],axis=1, inplace=True)

In [22]:
def preprocess_data(data):
    # Realiza el tratamiento de datos
    Record = pd.get_dummies(data['RecordType'])
    data = pd.concat([data, Record], axis=1)
    data.drop(['RecordType'], axis=1, inplace=True)
    
    return data
# merged_data = merged_data.drop(merged_data[(merged_data['Month'] < 6)].index)

In [23]:
joblib.dump(preprocess_data, 'preprocess_function.pkl')

['preprocess_function.pkl']

In [24]:
# merged_data['flight_route'] = merged_data['DestAirportID'].astype(str) + '_' + merged_data['AirportID'].astype(str)
# merged_data = pd.get_dummies(merged_data, columns=['flight_route'])
# merged_data.drop(['DestAirportID','AirportID'],axis=1, inplace=True)
# merged_data.head()

In [35]:
merged_data.to_csv('merged_data.csv', index=False)

# Paso 4 Modelado

In [26]:
#División del dataset en variable objetivo (Y) y variables explicativas (X)
X = merged_data.drop(columns =['DepDel15'],axis=1)
Y = merged_data['DepDel15']

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

In [39]:
y_test.to_csv('y_test.csv', index=False)

In [34]:
X_test.to_csv('merged_data.csv', index=False)

,Month,DayOfWeek,DestAirportID,AirportID,Day,Visibility,DryBulbCelsius,WetBulbCelsius,DewPointCelsius,RelativeHumidity,WindSpeed,WindDirection,StationPressure,Altimeter,AA,SP,SY-MT
570337,5,3,12264,14524,8,0.5,11.7,11.6,11.7,100.0,0.0,0.0,29.79,29.97,True,False,False
787003,5,7,10529,10821,19,7.0,19.0,18.2,18.0,93.0,8.0,90.0,29.96,30.13,False,True,False
450319,5,4,13495,12191,23,10.0,28.9,24.2,22.2,67.0,10.0,160.0,29.96,30.01,True,False,False
218667,4,3,11278,10397,10,10.0,22.8,16.2,11.7,50.0,11.0,180.0,28.87,29.96,True,False,False
545226,5,4,12953,11697,30,10.0,27.3,24.5,23.5,79.0,22.0,100.0,30.01,30.02,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1028858,6,3,11057,15304,26,10.0,23.9,23.8,23.9,100.0,0.0,0.0,30.05,30.06,True,False,False
308339,4,5,11433,13931,5,10.0,10.0,8.3,6.7,80.0,11.0,360.0,29.87,29.90,True,False,False
938786,6,7,14492,12339,23,7.0,21.7,20.5,20.0,90.0,9.0,140.0,29.19,30.06,True,False,False
259650,4,5,11066,13232,12,10.0,4.0,1.4,-2.0,65.0,18.0,260.0,29.15,29.81,False,True,False


In [29]:
### Entrenamiento de los modelos

In [30]:
#Importamos las librerias de sklearn con los diferentes algoritmos que queremos probar
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

### MODELADO

In [31]:
# Crea un modelo Gradient Boosting
model = GradientBoostingClassifier(
    n_estimators=100,
    max_depth=3,
    max_features='sqrt',
    loss='exponential',
    learning_rate=0.05,
    random_state=42
)

# Entrena el modelo en el conjunto de datos equilibrado
model.fit(X_train, y_train)

# Realiza predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Evalúa el modelo en el conjunto de prueba
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Imprime las métricas de evaluación
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

Accuracy: 0.79
Precision: 0.79
Recall: 0.01
F1 Score: 0.03


In [32]:
# Guardas el modelo en un archivo .pkl
joblib.dump(model, 'modelo_entrenado.pkl')

['modelo_entrenado.pkl']